In [2]:
import tensorflow as tf
from keras.models import Sequential

#Without Keras Autotuner

In [4]:
#define the model
#the parameters given for layers is changed based on hit and trails
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation ='softmax')
])

In [5]:
from keras.api._v2.keras import metrics
#compile model
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [6]:
#import dataset
from keras.datasets import fashion_mnist

#load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
#model training
model.fit(x_train, y_train, epochs= 5)

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 3.0601 - accuracy: 0.7029
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.7328 - accuracy: 0.7411
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.7051 - accuracy: 0.7562
Epoch 4/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.6952 - accuracy: 0.7597
Epoch 5/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.6918 - accuracy: 0.7601


In [8]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.5651 - accuracy: 0.8037


[0.5650598406791687, 0.8036999702453613]

#With Keras Autotuner

In [9]:
#install
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 10.5 MB/s eta 0:00:00


In [10]:
import kerastuner as kt

<ipython-input-10-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
#build model method
def model_builder(hp):
  model= tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = (28,28)))

  hp_units = hp.Int('units', min_value = 16, max_value = 512, step = 16)
  model.add(tf.keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(10))

  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [15]:
#usig Hyperband autotuner many tuners are available
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'Introduction_to_Keras_Autotuner')

In [16]:
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5
)

tuner.search(x_train,
             y_train,
             epochs = 50,
             validation_split = 0.2,
             callbacks =[stop_early])

Trial 30 Complete [00h 01m 19s]
val_accuracy: 0.007916666567325592

Best val_accuracy So Far: 0.23508332669734955
Total elapsed time: 00h 13m 07s


#Update the model with autotuner values

In [26]:
#define the model
#the parameters given for layers is changed based on hit and trails
updated_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation ='softmax')
])

In [27]:
from keras.api._v2.keras import metrics
#compile model
updated_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

In [28]:
#model training
updated_model.fit(x_train, y_train, epochs= 5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 2.6715 - accuracy: 0.4307
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.2834 - accuracy: 0.5057
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.1495 - accuracy: 0.5563
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.0970 - accuracy: 0.5817
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.0265 - accuracy: 0.6139


In [29]:
updated_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.8135 - accuracy: 0.7243


[0.813545823097229, 0.7243000268936157]